In [1]:
%load_ext autoreload
%autoreload 2

In [73]:
from wdtools import *
import webbrowser

In [3]:
revpath = inpath + '\GIS\ArcGIS Pro Project\DataReview\DataReview.gdb'

In [4]:
setID = 'Set003'

In [5]:
nm_to_add = 2143

In [6]:
wd = combine_wd_tables(setID=setID, nm_to_add=nm_to_add)

In [7]:
wd.columns

Index(['wetdet_delin_number', 'trsqq', 'parcel_id', 'address_location_desc',
       'city', 'county', 'site_name', 'site_desc', 'latitude', 'longitude',
       'Coord-Source', 'DocumentName', 'DecisionLink', 'is_batch_file',
       'status_name', 'received_date', 'response_date',
       'reissuance_response_date', 'project_id', 'site_id', 'record_ID',
       'notes', 'recyear', 'IDyear', 'missinglot'],
      dtype='object')

In [8]:
len(wd.wetdet_delin_number.unique())

433

In [9]:
matched = gpd.read_file(inpath + f'\\output\matched\matched_records_{setID}_edited.shp')

In [10]:
mapped0 = [lyr for lyr in fiona.listlayers(revpath) if (lyr not in [f'{setID}_wo_lot', f'{setID}_partial']) and ('L' not in lyr)]

In [11]:
partial = gpd.read_file(revpath, layer=f'{setID}_partial')

In [12]:
revise_single_partial_file('WD2019_0130')

,Shape_Length,Shape_Area,wdID,geometry
0,14604.540094,8.568226e+06,WD2019-0130,"MULTIPOLYGON (((710392.945 1475437.580, 710804..."


In [13]:
edited = ['WD2018-0261', 'WD2018-0422', 'WD2018-0635', 'WD2018-0642', 'WD2019-0048', 
          'WD2019-0369', 'WD2020-0507', 'WD2021-0536', 'WD2021-0590', 'WD2021-0700']

In [74]:
gdf, toCheck, digitized_nIDs, issueIDs = combine_matched_digitized(setID=setID, editedIDs=edited, nm_to_add=nm_to_add)

In [18]:
len(gdf.wdID.unique())

424

In [19]:
toCheck

[]

In [33]:
digitized_nIDs

119

In [44]:
editedIDs = edited

In [45]:
mapped0 = [lyr for lyr in fiona.listlayers(revpath) if (lyr not in [f'{setID}_wo_lot', f'{setID}_partial']) and ('L' not in lyr)]

In [46]:
matched = gpd.read_file(inpath + f'\\output\matched\matched_records_{setID}_edited.shp')

In [48]:
matched.crs

<Projected CRS: EPSG:2992>
Name: NAD83 / Oregon GIC Lambert (ft)
Axis Info [cartesian]:
- X[east]: Easting (foot)
- Y[north]: Northing (foot)
Area of Use:
- name: United States (USA) - Oregon.
- bounds: (-124.6, 41.98, -116.47, 46.26)
Coordinate Operation:
- name: Oregon GIC Lambert (International feet)
- method: Lambert Conic Conformal (2SP)
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [47]:
partial = gpd.read_file(revpath, layer=f'{setID}_partial')

In [50]:
partial = partial.to_crs(epsg=2992)

In [65]:
gdf = merge_single_partial_file(mapped0)

In [64]:
len(mapped0)

35

In [66]:
gdf.shape

(119, 4)

In [68]:
len(gdf.wdID.unique())

109

In [54]:
dat = gdf.append(partial, ignore_index=True)
edited_gdf = matched[matched.wdID.isin(editedIDs)]
edited_gdf = edited_gdf[['wdID', 'geometry']].dissolve('wdID')
edited_gdf.loc[:, 'wdID'] = edited_gdf.index
data1 = edited_gdf.append(dat[['wdID', 'geometry']], ignore_index=True)

In [56]:
matched_df = matched[['wdID', 'geometry']].dissolve('wdID')
matched_df.loc[:, 'wdID'] = matched_df.index
excluded = [wdID for wdID in data1.wdID.unique() if wdID in matched_df.wdID.unique()]
issues = pd.read_csv(os.path.join(inpath, "output", "to_review", f"{setID}_Mapping_Issues.csv"))
issueIDs = list(issues.wetdet_delin_number.unique())
matched_gdf = matched_df[~matched_df.wdID.isin(excluded+issueIDs)]
data2 = matched_gdf.append(data1, ignore_index=True)

In [57]:
wo_lot = gpd.read_file(revpath, layer=f'{setID}_wo_lot')

In [59]:
wo_lot = wo_lot.to_crs(epsg=2992)

In [60]:
wd = combine_wd_tables(setID=setID, nm_to_add=nm_to_add)
data3 = data2.append(wo_lot[['wdID', 'geometry']], ignore_index=True)
unmatchedIDs = [wdID for wdID in wd.wetdet_delin_number.unique() if wdID not in data3.wdID.unique()]
toCheck = [ID for ID in unmatchedIDs if ID not in issueIDs]
digitized_nIDs = len(editedIDs) + len(dat.wdID.unique()) + len(wo_lot.wdID.unique())

In [62]:
data3.shape

(509, 2)

In [61]:
data3.to_file(os.path.join(inpath, 'output', 'final', f'mapped_wd_{setID}.shp'))

In [ ]:
wdID = 'WD2021-0181'
url = wd.loc[wd.wetdet_delin_number == wdID, 'DecisionLink'].values[0]
webbrowser.open(url)

In [216]:
url

nan

In [221]:
wdID

'WD2021-0181'

In [222]:
wd.loc[wd.wetdet_delin_number == wdID, 'record_ID']

659    2803
Name: record_ID, dtype: int32

In [ ]:
extract_page_from_docLink(url=url, pageNm=4, wdID=wdID)

In [214]:
extract_page_from_locPath(filePath=r'C:\Users\clid1852\Downloads\WD20190060 AgencyDecision.pdf', #T:\DCProjects\EPA-WD\docs\
                          pageNm=8, wdID=wdID)

In [47]:
tomapID = matched[matched.notes.str.contains('Partial', case=False, na=False)].wdID.unique()